In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from portfolio_xml_tools import create_trade_from_term_sheet
from config_file import term_sheet_file


loader = PyPDFLoader(r'D:\Project_H\usd-inr-ccs-fixed-float.pdf')
pages = []
doc_data = ''
async for page in loader.alazy_load():
    pages.append(page)
    doc_data += page.page_content + '\n'


term_sheet_file.term_sheet_data = doc_data

test = create_trade_from_term_sheet.invoke({})


In [36]:
from testbench import foo
print(foo.term_sheet_data)
foo.term_sheet_data = "dummy string 200"
print(foo.term_sheet_data)

dummy string 2
dummy string 200


In [34]:
del foo

In [35]:
from testbench import foo
print(foo.term_sheet_data)


dummy string 2


In [12]:
from portfolio_xml_tools import create_trade_from_term_sheet

test = create_trade_from_term_sheet.invoke({"user_query": "Create a trade from given term sheet data.", "term_sheet_data": doc_data})


c:\Users\shrey\anaconda3\envs\ore_test\Lib\site-packages\pydantic\json_schema.py:2279: PydanticJsonSchemaWarning: Default value <class 'str'> is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\shrey\anaconda3\envs\ore_test\Lib\site-packages\pydantic\json_schema.py:2279: PydanticJsonSchemaWarning: Default value <class 'str'> is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [7]:
import uuid
from typing import Optional
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.constants import START, END
from langgraph.graph import StateGraph
from langgraph.types import interrupt, Command
from langgraph.prebuilt import create_react_agent
from ore_xml_tools import list_ore_tools as ore_xml_tools_list
from langchain.tools import tool
from typing import Literal
from langchain_core.messages.tool import ToolMessage
from llm_manager import llm
from ExtendedStatePlanExecute import AgentResponseSchema, PlanExecuteState

@tool
def ask_human(query: str) -> str:
    """Get human response to the given query."""
    human_response = interrupt({"text_to_revise": query})
    return ToolMessage(content=human_response)

agent = create_react_agent(llm,
    tools = [ask_human],
    )
class State(TypedDict):
    """The graph state."""

    foo: str
    human_value: Optional[str]
    """Human value will be updated using an interrupt."""

def main_node(state: State):
    state['foo'] = 'Hanuman'


def node(state: State):
    agent = create_react_agent(llm,
    tools = [ask_human],
    )
    test = agent.invoke([HumanMessage(content='Ask human about quantum finance.')])
    return test

def end_node(state: State):
    print(f"> Received an input from the interrupt: {state['human_value']}")
    return {}

builder = StateGraph(State)
builder.add_node("node", node)
builder.add_node("main_node", main_node)
builder.add_node("end_node", end_node)
builder.add_edge(START, "main_node")
builder.add_edge("main_node", "node")
builder.add_edge("node", "end_node")
builder.add_edge("end_node", END)

# A checkpointer must be enabled for interrupts to work!
checkpointer = MemorySaver()
graph = builder.compile(checkpointer=checkpointer)

config = {
    "configurable": {
        "thread_id": uuid.uuid4(),
    }
}

for chunk in graph.stream({"foo": "abc"}, config):
    print(chunk)
    if chunk.get('__interrupt__'):
        input_human = 30
        command = Command(resume=input_human)

for chunk in graph.stream(command, config):
    print(chunk)

{'main_node': None}


InvalidUpdateError: Expected dict, got [HumanMessage(content='Ask human about quantum finance.', additional_kwargs={}, response_metadata={})]
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE

In [23]:
import uuid
from typing import Optional
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.constants import START, END
from langgraph.graph import StateGraph
from langgraph.types import interrupt, Command
from langgraph.prebuilt import create_react_agent
from ore_xml_tools import list_ore_tools as ore_xml_tools_list
from langchain.tools import tool
from typing import Literal
from langchain_core.messages.tool import ToolMessage
from llm_manager import llm
from ExtendedStatePlanExecute import AgentResponseSchema, PlanExecuteState

@tool
def ask_human(query: str) -> str:
    """Get human response to get help on anything. frame your request in the given query."""
    human_response = interrupt({"text_to_revise": query})
    return ToolMessage(content=human_response)

# @tool
# def add_numbers(a: int, b: int) -> int:
#     """Add two numbers."""
#     return a + b

agent = create_react_agent(llm,
    tools = [ask_human],
    )

In [24]:
test = agent.invoke({'messages':[HumanMessage(content='add 2 and 3. what is circumference of a circle.')]})

In [29]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image, display


class State(TypedDict):
    input: str
    user_feedback: str


def step_1(state):
    print("---Step 1---")
    pass


def human_feedback(state):
    print("---human_feedback---")
    feedback = interrupt("Please provide feedback:")
    return {"user_feedback": feedback}


def step_3(state):
    print("---Step 3---")
    print(state["user_feedback"])
    pass


builder = StateGraph(State)
builder.add_node("step_1", step_1)
builder.add_node("human_feedback", human_feedback)
builder.add_node("step_3", step_3)
builder.add_edge(START, "step_1")
builder.add_edge("step_1", "human_feedback")
builder.add_edge("human_feedback", "step_3")
builder.add_edge("step_3", END)

# Set up memory
memory = MemorySaver()

# Add
graph = builder.compile(checkpointer=memory)

# Input
initial_input = {"input": "hello world"}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="updates"):
    print(event)
    print("\n")


# Continue the graph execution
for event in graph.stream(
    Command(resume="go to step 3!"), thread, stream_mode="updates"
):
    print(event)
    print("\n")

---Step 1---
{'step_1': None}


---human_feedback---
{'__interrupt__': (Interrupt(value='Please provide feedback:', resumable=True, ns=['human_feedback:28f04f6f-8791-f31f-fbd9-dcaabbe81d05'], when='during'),)}


---human_feedback---
{'human_feedback': {'user_feedback': 'go to step 3!'}}


---Step 3---
go to step 3!
{'step_3': None}




In [3]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.prebuilt import create_react_agent
from tools_utility import list_tools as common_tools_list
from ore_xml_tools import list_ore_tools as ore_xml_tools_list
from langchain.tools import tool
from react_agent_system_prompts import *
from langgraph.types import Command, interrupt
from typing import Literal
from llm_manager import llm
from config_file import f_path_in, f_path_out
from ExtendedStatePlanExecute import AgentResponseSchema, PlanExecuteState

# @tool
# def get_human_response(query: str) -> str:
#     """Get human response to the given query."""
#     human_response = interrupt({"text_to_revise": query})
#     return Co


all_tools_list_ore_xml = ore_xml_tools_list + common_tools_list



input_messages = {'messages': [SystemMessage(content=ore_xml_agent_system_prompt_content)]}
ore_xml_agent = create_react_agent(llm,
    tools = all_tools_list_ore_xml,
    response_format=AgentResponseSchema,
    interrupt_before=["tools"]
)

def ore_xml_agent_node_test(state: PlanExecuteState) -> Command[Literal["replanner"]]:
    messages_list = input_messages["messages"] + [HumanMessage(content='\n\nTask: ' + state["next_task"])] + [HumanMessage(content="Stopping criteria: " + state["stopping_criteria"])]
    messages = {'messages': messages_list}
    response = ore_xml_agent.invoke(messages)
    response_structured = response['structured_response']
    return Command(goto="replanner", update={"messages": [
                AIMessage(content=response_structured.summary.strip(), name="ore_xml_agent")
            ], 'past_steps': [(state["next_agent"], response_structured.summary, response_structured.status)]})



user_query = """get list of analytics in ore xml"""

# # test = []
for s in ore_xml_agent.stream(
    {"messages": [HumanMessage(content=user_query)],"input_path":f_path_in,"output_path":f_path_out,"user_query": user_query}):
    print(s)
    print("---")






{'agent': {'messages': [AIMessage(content=[{'text': "I'll help you retrieve the list of analytics in the ore.xml file. I'll use the `list_analytics` function to do this.", 'type': 'text'}, {'id': 'toolu_01XfYEr7GDEZhK6uoKErP23P', 'input': {}, 'name': 'list_analytics', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01Sj4QFSs7REHKhdVLbRYL1u', 'model': 'claude-3-5-haiku-20241022', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1982, 'output_tokens': 67}, 'model_name': 'claude-3-5-haiku-20241022'}, id='run-85154a46-0a08-4b3c-8719-b0e55976e727-0', tool_calls=[{'name': 'list_analytics', 'args': {}, 'id': 'toolu_01XfYEr7GDEZhK6uoKErP23P', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1982, 'output_tokens': 67, 'total_tokens': 2049, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})]}}
---
{'__interrupt__': ()}
---


> Received an input from the interrupt: some input from a human!!!
{'node': {'human_value': 'some input from a human!!!'}}


In [14]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver



class State(TypedDict):
    input: str
    user_feedback: str


def step_1(state):
    print("---Step 1---")
    pass


def human_feedback(state):
    print("---human_feedback---")
    feedback = interrupt("Please provide feedback:")
    return {"user_feedback": feedback}


def step_3(state):
    print("---Step 3---")
    pass


builder = StateGraph(State)
builder.add_node("step_1", step_1)
builder.add_node("human_feedback", human_feedback)
builder.add_node("step_3", step_3)
builder.add_edge(START, "step_1")
builder.add_edge("step_1", "human_feedback")
builder.add_edge("human_feedback", "step_3")
builder.add_edge("step_3", END)

# Set up memory
memory = MemorySaver()

# Add
graph = builder.compile(checkpointer=memory)

# Input
initial_input = {"input": "hello world"}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="updates"):
    print(event)
    print("\n")

---Step 1---
{'step_1': None}


---human_feedback---
{'__interrupt__': (Interrupt(value='Please provide feedback:', resumable=True, ns=['human_feedback:1842b1fa-954e-31b7-b1de-52448fd7278d'], when='during'),)}




In [15]:
# Continue the graph execution
for event in graph.stream(
    Command(resume="go to step 3!"), thread, stream_mode="updates"
):
    print(event)
    print("\n")

---human_feedback---
{'human_feedback': {'user_feedback': 'go to step 3!'}}


---Step 3---
{'step_3': None}




In [15]:
from langchain_core.messages import SystemMessage
SystemMessage(content="Testing").content

'Testing'